In [7]:
import matplotlib.pyplot as plt
import seaborn
from scipy import stats
from IPython.core.display import HTML, Latex, Markdown

## Clustering metrics
- Elbow method: Calculate within cluster sum of squared errors (WSS) for different $k$, and choose $k$ when WSS first starts to floor. (elbow point)
- Silhourette method: Denote $a$ the mean distance between a samples to all other points in the same class, and $b$ the mean distance between a sample and all other points in the next nearest cluster, 
$$a(i)=\frac{1}{|C_k|-1}\sum_{i\neq j}d(i, j),\quad i,j\in C_k$$
$$b(i)=\min\frac{1}{|C_l|}\sum_{j\in C_l}d(i, j),\quad i\in C_k,\ j\in C_l$$
$$s(i) = \frac{b(i)-a(i)}{\max[a(i), b(i)]},\quad s(i)=0,\ \rm{if}\ |C_k|=1$$
 - Silhourette score $-1\leq s(i) \leq 1$, close to 1 means data is appropriately clustered, close -1 means poorly clustered, 0 means data on the border of two clusters. $\bar s(k) = \sum_i s(i, k)$
 - But $s$ is generally higher for convex clusters than other concepts of clusters, such as density based clusters like those obtained through DBSCAN.
- Calinski-Harabaz index (Variance ration criteria):
 - the ratio of the sum of between-clusters dispersion and of inter-cluster dispersion for all clusters (where dispersion is defined as the sum of distances squared)
$$s = \frac{\mathrm{tr}(B_k)}{\mathrm{tr}(W_k)} \times \frac{n_E - k}{k - 1}$$
 - where $\mathrm{tr}(B_k)$ is trace of the between group dispersion matrix and $\mathrm{tr}(W_k)$ is the trace of the within-cluster dispersion matrix.
 - The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.
 - The score is fast to compute.
 - But index is generally higher for convex clusters than other concepts of clusters, such as density based clusters like those obtained through DBSCAN.
- If ground truth is known, can use adjusted rand index (ARI).
- If ground truth is known, can use mutual information based score.
- Metric for DBSCAN? `Moulavi, D., Jaskowiak, P. A., Campello, R. J., Zimek, A., & Sander, J. (2014, April). Density-based clustering validation. In Proceedings of the 2014 SIAM International Conference on Data Mining (pp. 839-847). Society for Industrial and Applied Mathematics.`

## Clustering

### Expection-Maximization
The Expectation-Maximization (EM) algorithm gives an efficient method at estimating the parameter $\theta$ through maximum likelihood estimation by repeatedly constructing a lower-bound on the likelihood (E-step) and optimizing that lower bound (M-step) as follows:
- E-step: Evaluate posterior probability $Q_i(z^{(i)}$ that each data point $x^{(i)}$ came from a particular cluster $z^{(i)}$
$$Q_i(z^{(i)})=P(z^{(i)}|x^{(i)};\theta)$$
- M-step: Use the posterior probabilities $Q_i(z^{(i)}$ as cluster specific weights on data points $x^{(i)}$ to re-estimate each cluster model as follows
$$\theta_i=\underset{\theta}{\textrm{argmax }}\sum_i\int_{z^{(i)}}Q_i(z^{(i)})\log\left(\frac{P(x^{(i)},z^{(i)};\theta)}{Q_i(z^{(i)})}\right)dz^{(i)}$$

### K-means
Denote $c^{(k)}$ the cluster data point $i$ and $\mu_j$ the center of cluster $j$, Algorithm
- Randomly initialize the cluster centroid $\mu_1,\mu_2,...,\mu_k\in\mathbb{R}^n$, repeats the following steps until convergence
 - $$c^{(i)}=\underset{j}{\textrm{arg min}}||x^{(i)}-\mu_j||^2$$
 - $$\mu_j=\frac{\sum_{i=1}^m1_{\{c^{(i)}=j\}}x^{(i)}}{\sum_{i=1}^m1_{\{c^{(i)}=j\}}}$$
 
 K-means is equivalent to the expectation-maximization algorithm with a small, all-equal, diagonal covariance matrix.

### Mini batch K-means
Uses mini-batches to reduce the computation time, while still attempting to optimise the same objective function.
- $m$ samples are drawn randomly from the dataset, to form a mini-batch. These are then assigned to the nearest centroid.
- The centroids are updated. In contrast to k-means, this is done on a per-sample basis. For each sample in the mini-batch, the assigned centroid is updated by taking the streaming average of the sample and all previous samples assigned to that centroid.

MiniBatchKMeans converges faster than KMeans, but the quality of the results is reduced. In practice this difference in quality can be quite small.

### Hierarchical clustering
It is a clustering algorithm with an hierarchical approach that build nested clusters in a successive manner. There are different types of hierarchical clustering algorithms that aims at optimizing different objective functions,
- Ward linkage: minimizes the sum of squared differences within all clusters. It is a variance-minimizing approach and in this sense is similar to the k-means objective function but tackled with an agglomerative hierarchical approach
- Average linkage: minimizes the average of the distances between all observations of pairs of clusters
- Complete linkage: minimizes the distance between the closest observations of pairs of clusters

### DBSCAN
Density based spatial clustering of applications with noise.

Algorithm description I
- Find points within eps distance of every point, and identify core points with more than minPoints neighbors
- Find the connected components of core points on the neighbor graph, ignore all non-core points
- Assign each non-core points to a near-by cluster if the cluster is an eps neighbor, otherwise noise.
- Features need to be normalized to make distance calculation meaningful

Algorithm description II
 - The algorithm proceeds by arbitrarily picking up point in the dataset (until all points have been visited).
 - If there are at least minPoints points within a radius of $\epsilon$ to the point then we consider all these points to be part of the same cluster.
- The clusters are then expanded by recursively repeating the neighborhood calculation for each neighboring point.
- The complexity of this algorithm is O(n2), where n is the number of points.
- With spatial indexing (kd-tree or r-tree), the complexity is $O(n\log(n))$

Hyper-parameters:
 - eps maximal distance/radius (distance or radius?): The idea is to calculate the average of the distances of every point to its k nearest neighbors. The value of k will be specified by the user and corresponds to MinPts. Next, plot these k-distances in ascending order. The aim is to determine the “knee”, which corresponds to the optimal eps parameter.
 - minPoints (minimal number of points including self): typically $\geq D+1$ or $2D$, where $D$ is dimension of the dataset. May be necessary to chose larger values for very large data set, or noisy data.
 - sometimes distance function other than Euclidean distance can be used

Advantages:
- does not need to specify the number of clusters apriori
- can find arbitrarily shaped clusters. minPoints parameter can reduce single link effect
- is robust to outliers
- designed for use with databases with accelerated region (range) queries
- eps and minPoints can be set by a domain expert, if data is well understood

Disadvantages
- not entirely determinstic: border points can be reached from more than one cluster
- commonly used distance measure such as Euclidean distance would not perform well for high-dimensional data. (Curse of dimensionality)
- can not cluster dataset well with large difference in densities
- if data and scale are not well understood, choose a meaningful distance threshold can be difficult

### Gaussian mixture
A Gaussian mixture model is a probabilistic model that assumes all the data points are generated from a mixture of a finite number of Gaussian distributions with unknown parameters. One can think of mixture models as generalizing k-means clustering to incorporate information about the covariance structure of the data as well as the centers of the latent Gaussians.
- The GaussianMixture object implements the expectation-maximization (EM) algorithm for fitting mixture-of-Gaussian models. It can also draw confidence ellipsoids for multivariate models, and compute the Bayesian Information Criterion to assess the number of clusters in the data.
- Fast to learn
- When one has insufficiently many points per mixture, estimating the covariance matrices becomes difficult, and the algorithm is known to diverge and find solutions with infinite likelihood unless one regularizes the covariances artificially.

## Dimension reduction

### Principal component analysis
Find the variance maximization direction to project the data. In other words, find the k-dimension subspace of N-dim data that preserve the most energy.
- Normalize the data to have mean 0 and std of 1. Is it really needed
- Compute $\displaystyle\Sigma=\frac{1}{m}\sum_{i=1}^mx^{(i)}{x^{(i)}}^T$, which is symmetric with real eigenvalues (semi positive), $\mathbb{R}^{n\times n}$
- Compute its $k$ principal eigenvectors $e_k$, corresponds to the $k$ largest igenvalues
- Projec the data on span $(e_1,...e_k)$

Can just use singular value decomposition (SVD).

### Independent component analysis
We assume that our data $x$ has been generated by the n-dimensional source vector $s=(s_1,...,s_n)$, where $s_i$ are independent random variables, via a mixing and non-singular matrix $A$ as follows:
$$x=As$$
The goal is to find unmixing matrix $W=A^{-1}$, solution